# Maestría en "Analítica de datos"
---
Nombre: Luis Fernando Castellanos Guarin
Código: XXXX
Fecha: 2025 agosto 28
---
Descripción: este libro trabaja ejercicios de "analítica/ETL/estadística descriptiva"

# 1.Cargar librerias

In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.carga/obtención de **información**

In [ ]:
file_path_csv="/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/DataSets_TMP/Población_Base_de_Datos_Única_de_Afiliados_BDUA_del_régimen_contributivo.csv"
try:
  df=pd.read_csv(file_path_csv)
except FileNotFoundError:
  print(f"El archivo {file_path_csv} no fue encontrado.")
except Exception as e:
  print(f"Ocurrió un error al leer el archivo: {e}")

In [ ]:
pd.set_option("display.max_columns", None)
display(df.head())

,Género,Grupo etario,Código de la entidad,Nombre de la entidad,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros
0,Masculino,45 a 50,ESSC24,COOSALUD EPS S.A. -CM,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana-Cabecera Municipal,MAGDALENA,PIVIJAY,NaN,NaN,6
1,Femenino,> 75,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,BOLIVAR,CALAMAR,1,POBLACIÓN CON SISBEN,1
2,Masculino,15 a 19,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Protección Laboral C,NO APLICA,Urbana,ARAUCA,ARAUCA,N,VÍCTIMAS DEL CONFLICTO ARMADO INTERNO,1
3,Femenino,15 a 19,EPS017,EPS FAMISANAR S.A.S.,Contributivo,BENEFICIARIO,Activo,ESTUDIANTE,Urbana-Cabecera Municipal,CUNDINAMARCA,MOSQUERA,NaN,NaN,2
4,Masculino,45 a 50,ESSC62,ASMET SALUD EPS S.A.S. -CM,Contributivo,COTIZANTE,Activo,NO APLICA,Rural - Dispersal,CAUCA,SUAREZ,1,POBLACIÓN CON SISBEN,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656598 entries, 0 to 656597
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Género                      656598 non-null  object
 1   Grupo etario                656598 non-null  object
 2   Código de la entidad        656598 non-null  object
 3   Nombre de la entidad        656598 non-null  object
 4   Régimen                     656598 non-null  object
 5   Tipo de afiliado            656598 non-null  object
 6   Estado del afiliado         656598 non-null  object
 7   Condición del beneficiario  656598 non-null  object
 8   Zona de Afiliación          656598 non-null  object
 9   Departamento                656598 non-null  object
 10  Municipio                   656598 non-null  object
 11  tps_nvl_ssb_id              369349 non-null  object
 12  Nivel del Sisbén            369250 non-null  object
 13  Cantidad de registros       6

In [ ]:
df.describe()

,Género,Grupo etario,Código de la entidad,Nombre de la entidad,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros
count,656598,656598,656598,656598,656598,656598,656598,656598,656598,656598,656598,369349,369250,656598
unique,2,12,31,31,1,3,2,3,6,33,1035,4,19,3034
top,Masculino,19 a 45,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,ANTIOQUIA,BOGOTA D.C.,1,POBLACIÓN CON SISBEN,1
freq,332480,156230,278042,278042,656598,338182,613592,629227,298692,83198,6251,143052,242862,285920


# 3.ETL

In [ ]:
# Convert 'Cantidad de registros' to numeric, coercing errors, then to integer
df['Cantidad de registros'] = pd.to_numeric(df['Cantidad de registros'], errors='coerce')
df['Cantidad de registros'] = df['Cantidad de registros'].fillna(0).astype(int)
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656598 entries, 0 to 656597
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Género                      656598 non-null  object
 1   Grupo etario                656598 non-null  object
 2   Código de la entidad        656598 non-null  object
 3   Nombre de la entidad        656598 non-null  object
 4   Régimen                     656598 non-null  object
 5   Tipo de afiliado            656598 non-null  object
 6   Estado del afiliado         656598 non-null  object
 7   Condición del beneficiario  656598 non-null  object
 8   Zona de Afiliación          656598 non-null  object
 9   Departamento                656598 non-null  object
 10  Municipio                   656598 non-null  object
 11  tps_nvl_ssb_id              369349 non-null  object
 12  Nivel del Sisbén            656598 non-null  object
 13  Cantidad de registros       6

None

In [ ]:
columnas_a_convertir =["Departamento","Municipio","Código de la entidad","Nombre de la entidad","Nivel del Sisbén"]
for columna in columnas_a_convertir:
  if columna in df.columns:
    df[columna]=df[columna].astype(str)


#4.estadísticas descriptivas

In [ ]:
df['Género'].value_counts()

,count
Género,
Masculino,332480
Femenino,324118


In [ ]:
df_eps= df.groupby('Nombre de la entidad')['Cantidad de registros'].sum()
#calcular porcentaje
total_registros = df['Cantidad de registros'].sum()
entidad_porcentaje = (df_eps / total_registros) * 100
resultado_df = pd.DataFrame({'Cantidad de Registros': df_eps, 'Porcentaje': entidad_porcentaje})
resultado_df = resultado_df.sort_values(by='Porcentaje', ascending=False)
display(resultado_df.head())
#display(df_eps.head())

,Cantidad de Registros,Porcentaje
Nombre de la entidad,,
NUEVA EPS S.A.,3191312,37.240613
SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A.,1026566,11.979383
ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.,956870,11.166074
EPS SURAMERICANA S.A.,639941,7.467711
EPS FAMISANAR S.A.S.,619655,7.230986


# 5.creación de BD SQLITE y tablas

In [ ]:
db_path="/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/DataSets_TMP/SQLITE/dbua.db"
db_dir =os.path.dirname(db_path)
if os.path.exists(db_dir):
  print(f"El directorio {db_dir} existe.")
else:
  os.makedirs(db_dir)
  print(f"El directorio {db_dir} fue creado.")

El directorio /content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/DataSets_TMP/SQLITE existe.


In [ ]:
try:
  conn=sqlite3.connect(db_path)
  cursor=conn.cursor()          #objeto que va a interactiar con la bd (dml/ddl)
  print("conectado")
except sqlite3.Error as e:
  print(f"Error al conectar a la base de datos: {e}")


conectado


## 5.1 creando tablas maestras

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Departamento (
    Id_Departamento INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre_Departamento TEXT UNIQUE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Sisben (
    Id_Sisben INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre_Sisben TEXT UNIQUE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Eps (
    codigo_Eps TEXT PRIMARY KEY,
    Nombre_Eps TEXT UNIQUE NOT NULL,
    regimen TEXT
)
''')

## 5.2 llenado de tablas maestras

In [ ]:
#funcion para cargar datos en tablas maestras (departamento, sisben, eps)
def cargar_datos_a_tablas_maestras(df,db_path,columna, tabla_destino):
  conn= sqlite3.connect(db_path)
  cursor=conn.cursor()
  valores_unicos =df[columna].unique()
  for valor in valores_unicos:
    if(pd.notna(valor)): #ignorar valores NaN
      try:
        cursor.execute(f"INSERT OR IGNORE INTO {tabla_destino} (Nombre_{tabla_destino.replace('_','')}) VALUES (?)", (valor,))
      except sqlite3.Error as e:
        print(f"Error al insertar en la tabla {tabla_destino}: {e}")
        pass
  conn.commit()
  conn.close()

In [ ]:
cargar_datos_a_tablas_maestras(df,db_path,"Departamento","Departamento")
cargar_datos_a_tablas_maestras(df,db_path,"Nivel del Sisbén","Sisben")

In [ ]:
def revisar_contenido_de_una_tabla(tabla_nombre, whereColumna='',whereValor='', limit=10, order_by_columna=None, order_asc=True):
  conn=sqlite3.connect(db_path)
  cursor=conn.cursor()
  cursor.execute("SELECT COUNT(*) FROM "+tabla_nombre)
  total_registros=cursor.fetchone()[0]
  print(f"Total de registros en la tabla {tabla_nombre}: {total_registros}")
  sql="SELECT * FROM "+tabla_nombre
  params = ()
  if len(whereColumna)>0:
    sql+=" WHERE "+whereColumna+"=?"
    params = (whereValor,)
  if(order_by_columna):
    orderbyTipo="ASC" if order_asc else "DESC"
    sql+=f" ORDER BY {order_by_columna} {orderbyTipo}"
  sql+=" LIMIT "+str(limit)
  print(sql)
  cursor.execute(sql, params)
  resultados=cursor.fetchall()
  for fila in resultados:
    print(fila)
  conn.close()

In [ ]:
revisar_contenido_de_una_tabla("Departamento",'','',100,'Id_departamento')

Total de registros en la tabla Departamento: 33
SELECT * FROM Departamento ORDER BY Id_departamento ASC LIMIT 100
(1, 'MAGDALENA')
(2, 'BOLIVAR')
(3, 'ARAUCA')
(4, 'CUNDINAMARCA')
(5, 'CAUCA')
(6, 'VALLE')
(7, 'CALDAS')
(8, 'CASANARE')
(9, 'TOLIMA')
(10, 'NORTE DE SANTANDER')
(11, 'BOYACA')
(12, 'CAQUETA')
(13, 'ANTIOQUIA')
(14, 'CESAR')
(15, 'CORDOBA')
(16, 'QUINDIO')
(17, 'HUILA')
(18, 'AMAZONAS')
(19, 'META')
(20, 'NARINO')
(21, 'SUCRE')
(22, 'SANTANDER')
(23, 'RISARALDA')
(24, 'ATLANTICO')
(25, 'BOGOTA D.C.')
(26, 'VAUPES')
(27, 'LA GUAJIRA')
(28, 'SAN ANDRES')
(29, 'PUTUMAYO')
(30, 'CHOCO')
(31, 'VICHADA')
(32, 'GUAINIA')
(33, 'GUAVIARE')


In [ ]:
revisar_contenido_de_una_tabla("Sisben",'','',100,'nombre_sisben')

Total de registros en la tabla Sisben: 20
SELECT * FROM Sisben ORDER BY nombre_sisben ASC LIMIT 100
(14, 'ADULTO MAYOR EN CENTROS DE PROTECCIÓN')
(4, 'COMUNIDADES INDÍGENAS')
(8, 'HABITANTE DE LA CALLE')
(10, 'LOS VOLUNTARIOS ACREDITADOS Y ACTIVOS DE LA DEFENSA CIVIL COLOMBIANA CRUZ ROJA COLOMBIANA Y CUERPOS DE BOMBEROS')
(20, 'MENORES DESVINCULADOS DEL CONFLICTO ARMADO BAJO LA PROTECCIÓN DEL ICBF.')
(16, 'MIGRANTES COLOMBIANOS REPATRIADOS QUE HAN RETORNADO VOLUNTARIAMENTE AL PAÍS O HAN SIDO DEPORTADOS O EXPULSADOS DE TERRITORIO EXTRANJERO.')
(7, 'NIÑOS-NIÑAS-ADOLESCENTES Y JÓVENES EN PROCESO ADMINISTRATIVO PARA EL RESTABLECIMIENTO DE SUS DERECHOS')
(19, 'PERSONAS CON DISCAPACIDAD EN CENTROS DE PROTECCIÓN')
(12, 'PERSONAS EN PRISIÓN DOMICILIARIA A CARGO DEL INPEC.')
(17, 'PERSONAS QUE DEJEN DE SER MADRES COMUNITARIAS.')
(2, 'POBLACIÓN CON SISBEN')
(6, 'POBLACIÓN DESMOVILIZADA Y/O MIEMBROS DEL GRUPO ARMADO ILEGAL QUE CELEBREN ACUERDOS DE PAZ CON EL GOBIERNO NACIONAL')
(18, 'POBLACIÓN IN

## 5.3 tabla hija "municipio"

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Municipio (
    Id_Municipio INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre_Municipio TEXT NOT NULL,
    Id_Departamento INTEGER,
    FOREIGN KEY (Id_Departamento) REFERENCES Departamento(Id_Departamento),
    UNIQUE (Nombre_Municipio, Id_Departamento)
)
''')
conn.commit()
conn.close()

In [ ]:
#para cada municipio buscarle su departamento
def cargar_municipios(df,pd_path):
  conn= sqlite3.connect(db_path)
  cursor=conn.cursor()
  tot_creados=0
  for index,row in df.iterrows():
    departamento  =row['Departamento']
    municipio     =row['Municipio']
    cursor.execute("SELECT Id_Departamento FROM Departamento WHERE Nombre_Departamento=?", (departamento,))
    departamento_id=cursor.fetchone()
    if departamento_id:
      departamento_id=departamento_id[0]
      try:
        cursor.execute("INSERT OR IGNORE INTO Municipio (Nombre_Municipio, Id_Departamento) VALUES (?, ?)", (municipio, departamento_id))
        tot_creados+=1
        print(f"creado {tot_creados} => {municipio} en el dpto {departamento} ")
      except sqlite3.sqlite3.IntegrityError:
        pass
    else:
      print(f"ERRROR: Departamento {departamento} para el municipio {municipio} no encontrado")
  conn.commit()
  conn.close()

In [ ]:
cargar_municipios(df,db_path)

In [ ]:
revisar_contenido_de_una_tabla("Municipio",'','',10,'nombre_Municipio')

Total de registros en la tabla Municipio: 1122
SELECT * FROM Municipio ORDER BY nombre_Municipio ASC LIMIT 10
(804, 'ABEJORRAL', 13)
(746, 'ABREGO', 10)
(8040, 'ABRIAQUI', 13)
(1010, 'ACACIAS', 19)
(855, 'ACANDI', 30)
(3186, 'ACEVEDO', 17)
(1367, 'ACHI', 2)
(205, 'AGRADO', 17)
(2650, 'AGUA DE DIOS', 4)
(96, 'AGUACHICA', 14)


## 5.4 cargar EPS

In [ ]:
def cargar_datos_eps(df,db_path,regimen="Contributivo"):
 conn= sqlite3.connect(db_path)
 cursor=conn.cursor()
 total_creados=0
 valores_unicos = df[['Código de la entidad','Nombre de la entidad']].drop_duplicates()
 for index,row in valores_unicos.iterrows():
  codigo_eps= row['Código de la entidad']
  nombre_eps= row['Nombre de la entidad']
  try:
    cursor.execute("INSERT OR IGNORE INTO Eps (codigo_Eps, Nombre_Eps, Regimen) VALUES (?, ?,?)", (codigo_eps, nombre_eps,regimen))
    total_creados+=1
    print(f"creado {total_creados} => {nombre_eps} con el código {codigo_eps}")
  except sqlite3.IntegrityError as e:
    print(f"No fue posible crear la EPS {nombre_eps} con el código {codigo_eps} due to {e}")
    pass
 conn.commit()
 conn.close()

In [ ]:
cargar_datos_eps(df,db_path)

creado 1 => COOSALUD EPS S.A. -CM con el código ESSC24
creado 2 => NUEVA EPS S.A. con el código EPS037
creado 3 => EPS FAMISANAR S.A.S. con el código EPS017
creado 4 => ASMET SALUD EPS S.A.S. -CM con el código ESSC62
creado 5 => COOSALUD EPS S.A. con el código EPS042
creado 6 => CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR con el código EPS008
creado 7 => NUEVA EPS S.A. -CM con el código EPS041
creado 8 => ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S. con el código EPS005
creado 9 => SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A. con el código EPS002
creado 10 => ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS con el código EPS048
creado 11 => EPS SURAMERICANA S.A. con el código EPS010
creado 12 => ENTIDAD PROMOTORA DE SALUD MALLAMAS EPSI -CM con el código EPSIC5
creado 13 => ALIANZA MEDELLIN ANTIOQUIA EPS S.A.S. "SAVIA SALUD EPS" -CM con el código EPS040
creado 14 => ASOCIACION MUTUAL SER EMPRESA SOLIDA

In [ ]:
revisar_contenido_de_una_tabla("Municipio",'','',10,'nombre_Municipio')

## 5.5 crear tabla nieta "detalle"
---


In [ ]:
conn= sqlite3.connect(db_path)
cursor=conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS Detalle (
    Id_Detalle INTEGER PRIMARY KEY AUTOINCREMENT,
    Id_Municipio INTEGER,
    Id_Sisben INTEGER,
    codigo_Eps TEXT,
    Genero TEXT,
    Grupo_etario TEXT,
    Tipo_afiliado TEXT,
    Estado_afiliado TEXT,
    Condicion_beneficiario TEXT,
    Regimen TEXT,
    Zona_afiliacion TEXT,
    Cantidad_registros INTEGER,
    FOREIGN KEY (Id_Municipio) REFERENCES Municipio(Id_Municipio),
    FOREIGN KEY (Id_Sisben) REFERENCES Sisben(Id_Sisben),
    FOREIGN KEY (codigo_Eps) REFERENCES Eps(codigo_Eps)
)
''')
conn.commit()
conn.close()

In [ ]:
def cargar_detalle(df,db_path):
  conn= sqlite3.connect(db_path)
  cursor=conn.cursor()
  total_creados=0
  for index,row in df.iterrows():
    municipio   =row['Municipio']
    departamento=row['Departamento']
    #--------para obtener el id del sisben
    if (row['Nivel del Sisbén'] is None):
      id_sisben=0
    else:
      sisben=row['Nivel del Sisbén']
      cursor.execute("SELECT Id_Sisben FROM Sisben WHERE Nombre_Sisben=?", (sisben,))
      sisben_id=cursor.fetchone()
      if sisben_id:
        id_sisben=sisben_id[0]
      else:
        print(f"ERRROR: Sisben {sisben} para el municipio {municipio} no encontrado")
    #----------para obtener id del municipio
    cursor.execute("SELECT Id_Municipio FROM Municipio WHERE Nombre_Municipio=? AND Id_Departamento=(SELECT Id_Departamento FROM Departamento WHERE Nombre_Departamento=?)", (municipio,departamento))
    municipio_id=cursor.fetchone()
    if municipio_id:
      municipio_id=municipio_id[0]
    else:
      print(f"ERRROR: Municipio {municipio} para el departamento {departamento} no encontrado")
    #--------------guarde cada registro en la tabla---
    try:
      cursor.execute('''
      INSERT INTO Detalle (Id_Municipio, Id_Sisben, codigo_Eps, Genero, Grupo_etario, Tipo_afiliado, Estado_afiliado, Condicion_beneficiario, Regimen, Zona_afiliacion, Cantidad_registros)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                    (municipio_id,
                      id_sisben,
                      row['Código de la entidad'],
                      row['Género'],
                      row['Grupo etario'],
                      row['Tipo de afiliado'],
                      row['Estado del afiliado'],
                      row['Condición del beneficiario'],
                      row['Régimen'],
                      row['Zona de Afiliación'],
                      row['Cantidad de registros']
                    ))
      total_creados+=1
      print(f"creado {total_creados}")
    except sqlite3.IntegrityError as e:
      print(f"No fue posible crear el registro due to {e}")
      pass
  conn.commit()
  conn.close()



In [ ]:
cargar_detalle(df,db_path)

In [ ]:
revisar_contenido_de_una_tabla("Detalle",'','',10)

Total de registros en la tabla Detalle: 656598
SELECT * FROM Detalle LIMIT 10
(1, 1, 1, 'ESSC24', 'Masculino', '45 a 50', 'COTIZANTE', 'Activo', 'NO APLICA', 'Contributivo', 'Urbana-Cabecera Municipal', 6)
(2, 2, 2, 'EPS037', 'Femenino', '> 75', 'COTIZANTE', 'Activo', 'NO APLICA', 'Contributivo', 'Urbana', 1)
(3, 3, 3, 'EPS037', 'Masculino', '15 a 19', 'COTIZANTE', 'Protección Laboral C', 'NO APLICA', 'Contributivo', 'Urbana', 1)
(4, 4, 1, 'EPS017', 'Femenino', '15 a 19', 'BENEFICIARIO', 'Activo', 'ESTUDIANTE', 'Contributivo', 'Urbana-Cabecera Municipal', 2)
(5, 5, 2, 'ESSC62', 'Masculino', '45 a 50', 'COTIZANTE', 'Activo', 'NO APLICA', 'Contributivo', 'Rural - Dispersal', 1)
(6, 6, 3, 'EPS042', 'Femenino', '55 a 60', 'COTIZANTE', 'Activo', 'NO APLICA', 'Contributivo', 'Rural', 1)
(7, 7, 3, 'EPS037', 'Masculino', '15 a 19', 'COTIZANTE', 'Activo', 'NO APLICA', 'Contributivo', 'Urbana', 8)
(8, 8, 1, 'EPS008', 'Femenino', '55 a 60', 'BENEFICIARIO', 'Protección Laboral C', 'NO APLICA', 'Co

# 6.Integrar con tutelas radicadas

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
url_forder_csv="/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/DataSets_TMP/tutelas_2021_al_2025"
listado_archivos= [f for f in os.listdir(url_forder_csv) if f.endswith(".csv")]
print(listado_archivos)
columnas_nombre=[
    "clasificacion",
    "num_expediente",
    "fecha_Radica",
    "demandante",
    "demandado",
    "primera_instancia",
    "segunda_instancia",
    "num_23Digitos",
    "providencia",
    "fechaSentencia"
]
df_tutelas=pd.DataFrame(columns=columnas_nombre)
display (df_tutelas)

['2025.csv', '2024.csv', '2023.csv', '2021.csv', '2022.csv']


,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia


In [ ]:
def cargar_data_desde_archivo_csv(ruta_archivo):
  try:
      df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')
      #agregarle nombre de columnas al df creado
      df_temporal.columns=columnas_nombre
      if (len(df_temporal.columns)==len(columnas_nombre)):
        print(f" archivo {os.path.basename(ruta_archivo)} cargada exitosamente")
        return df_temporal
      else:
        print(f"archivo no trabajado {os.path.basename(ruta_archivo)} no tiene 10 columnas")
        return None

  except Exception as e:
      print(f"Error al procesar el archivo {os.path.basename(ruta_archivo)}: {str(e)}")
      return None

In [ ]:
df_list=[]  #lista de df resultante de los datos de cada uno de los CSV
with ThreadPoolExecutor() as executor:
  future_to_file= {executor.submit(cargar_data_desde_archivo_csv,os.path.join(url_forder_csv,archivo_nombre)):archivo_nombre for archivo_nombre in listado_archivos}
  for future in as_completed(future_to_file):
    df=future.result()
    if df is not None:
      df_list.append(df)
#--------una vez termine de cargar todos los CSV combinelos en el dataframa df_tutelas
if df_list:
  df_tutelas= pd.concat(df_list,ignore_index=True)
  # reemplazar las palabras "NULL" por none
  df_tutelas.replace("NULL",None,inplace=True)
  display(df_tutelas.head())


 archivo 2021.csv cargada exitosamente


/tmp/ipython-input-3430010226.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')


 archivo 2022.csv cargada exitosamente
 archivo 2025.csv cargada exitosamente
 archivo 2023.csv cargada exitosamente
 archivo 2024.csv cargada exitosamente


,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia
0,ï»¿Acciones de Tutela,T8531729,2021-12-16 00:00:00,VAZQUEZ CUADROS BAYRON EMILIO Y OTROS EN REPRE...,SOCIEDAD DE ACTIVOS ESPECIALES S.A.E. S.A.S.,"BOGOTA,CUNDINAMARCA, TRIBUNAL SUPERIOR DE BOGO...",--,--,--,NaN
1,Acciones de Tutela,T8531726,2021-12-16 00:00:00,ESCORCIA JORGE ARMANDO,ALCALDIA MUNICIPAL DE SANTA LUCIA,"SANTA LUCIA,ATLANTICO, JUZGADO 1 PROMISCUO MUN...","SABANALARGA,ATLANTICO, JUZGADO 2 PROMISCUO DEL...",--,--,NaN
2,Acciones de Tutela,T8531728,2021-12-16 00:00:00,ARCINIEGAS MEDRANO CLEMENCIA,COLPENCIONES Y OTRO,"BARRANQUILLA,ATLANTICO, JUZGADO 11 LABORAL DEL...","BARRANQUILLA,ATLANTICO, TRIBUNAL SUPERIOR SALA...",--,T-274/22,2022-07-28 00:00:00.000
3,Acciones de Tutela,T8531723,2021-12-16 00:00:00,SERRANO ROJAS CAMILO ANDRES,SECRETARIA DE MOVILIDAD DE CAQUEZA,"CAQUEZA,CUNDINAMARCA, JUZGADO 1 PROMISCUO MUNI...",--,--,--,NaN
4,Acciones de Tutela,T8531727,2021-12-16 00:00:00,GOMEZ QUINTERO EDILSA MARIA,CAJACOPI EPS Y OTRO,"CURUMANI,CESAR, JUZGADO PROMISCUO MUNICIPAL","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN


In [ ]:
df_tutelas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3371223 entries, 0 to 3371222
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   clasificacion      object
 1   num_expediente     object
 2   fecha_Radica       object
 3   demandante         object
 4   demandado          object
 5   primera_instancia  object
 6   segunda_instancia  object
 7   num_23Digitos      object
 8   providencia        object
 9   fechaSentencia     object
dtypes: object(10)
memory usage: 257.2+ MB


## 6.1 extraer los expedientes donde los demandados "eps"

In [ ]:
df_tutelas_eps=df_tutelas[df_tutelas['demandado'].str.contains("EPS",na=False)]
display(df_tutelas_eps.head())
df_tutelas_eps.count()

,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia
4,Acciones de Tutela,T8531727,2021-12-16 00:00:00,GOMEZ QUINTERO EDILSA MARIA,CAJACOPI EPS Y OTRO,"CURUMANI,CESAR, JUZGADO PROMISCUO MUNICIPAL","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN
10,Acciones de Tutela,T8531722,2021-12-16 00:00:00,ARAGON CORREA ANDREA AO DE NUÃEZ BARACALDO KE...,CONVIDA EPS T OTRO,"CAQUEZA,CUNDINAMARCA, JUZGADO 1 PROMISCUO MUNI...",--,--,--,NaN
17,Acciones de Tutela,T8531705,2021-12-16 00:00:00,QUIROZ RODRIGUEZ LIDIS ESTHER,LA NUEVA EPS Y OTROS,"PAILITAS,CESAR, JUZGADO PROMISCUO MUNICIPAL","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN
21,Acciones de Tutela,T8531702,2021-12-16 00:00:00,HERNANDEZ GOMEZ SEBASTIAN,COOSALUD EPS,"CHIGORODO,ANTIOQUIA, JUZGADO 1 PROMISCUO MUNIC...","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN
25,Acciones de Tutela,T8531700,2021-12-16 00:00:00,ROZO PARRA WILMAR,CAPITAL SALUD EPS,"ACACIAS,META, JUZGADO 1 PROMISCUO MUNICIPAL",--,--,--,NaN


,0
clasificacion,848569
num_expediente,848569
fecha_Radica,848569
demandante,848569
demandado,848569
primera_instancia,848568
segunda_instancia,545324
num_23Digitos,848569
providencia,848568
fechaSentencia,167


## 6.2 Extraer las EPS dbua.db (sqlite)

In [ ]:
conn= sqlite3.connect(db_path)
df_eps=pd.read_sql_query("SELECT * FROM Eps",conn)
conn.close()
display(df_eps.head())

,codigo_Eps,Nombre_Eps,regimen
0,ESSC24,COOSALUD EPS S.A. -CM,Contributivo
1,EPS037,NUEVA EPS S.A.,Contributivo
2,EPS017,EPS FAMISANAR S.A.S.,Contributivo
3,ESSC62,ASMET SALUD EPS S.A.S. -CM,Contributivo
4,EPS042,COOSALUD EPS S.A.,Contributivo


## 6.3 contar las tutelas en contra de cada EPS (df_eps)

In [ ]:
df_eps['total_tutelas']=0
demandados_encontrados =set()
for index,row in df_eps.iterrows():
  eps_nombre=row['Nombre_Eps']
  matching_tutelas = df_tutelas_eps[df_tutelas_eps['demandado'].str.contains(eps_nombre,na=False,case=False)]  #mejorar el código "NUEVA EPS"
  total_tutelas=matching_tutelas.shape[0]
  #df_eps.at[index,'total_tutelas']=total_tutelas
  df_eps.loc[index,'total_tutelas']=total_tutelas
  demandados_encontrados.update(matching_tutelas['demandado'].tolist())

demandados_no_encontrados=set(df_tutelas_eps['demandado'].unique())-demandados_encontrados
print(f"EPS demandadas encontrados: {len(demandados_encontrados)}")
print(f"EPS demandadas NO encontradas: {len(demandados_no_encontrados)}")
for demandado in demandados_no_encontrados:
  print(demandado)

In [ ]:
display(df_eps.head())

,codigo_Eps,Nombre_Eps,regimen,total_tutelas
0,ESSC24,COOSALUD EPS S.A. -CM,Contributivo,1
1,EPS037,NUEVA EPS S.A.,Contributivo,3907
2,EPS017,EPS FAMISANAR S.A.S.,Contributivo,212
3,ESSC62,ASMET SALUD EPS S.A.S. -CM,Contributivo,0
4,EPS042,COOSALUD EPS S.A.,Contributivo,320


# ....MEJORAR EL CÓDIGO para que encuentre las diversas variaciones de los nombres de las EPS